This notebook is adapted from [this one](https://github.com/dataflowr/notebooks/blob/archive-2020/Notebooks/02_basics_pytorch.ipynb) written by [Marc Lelarge](https://www.di.ens.fr/~lelarge/), and from the tutorials [What is PyTorch?](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#) and [Autograd: Automatic Differentiation](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py).

You can have access to more tutorials [here](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html).

When you have questions, try as much as possible to find the answer by looking at [PyTorch documentation](https://pytorch.org/docs/stable/index.html). You need to learn to work with the documentation.

In [ ]:
%matplotlib inline


# What is PyTorch?

It’s a Python-based scientific computing package targeted at two sets of
usage:

-  A replacement for NumPy to use the power of GPUs
-  A deep learning research platform that provides maximum flexibility
   and speed

## Tensors

Tensors are similar to NumPy’s ndarrays, with the addition being that
Tensors can also be used on a GPU to accelerate computing.



In [ ]:
import torch

Construct a 5x3 matrix, uninitialized:



<div class="alert alert-info"><h4>Note</h4><p>An uninitialized matrix is declared,
    but does not contain definite known
    values before it is used. When an
    uninitialized matrix is created,
    whatever values were in the allocated
    memory at the time will appear as the initial values.</p></div>



In [ ]:
x = torch.empty(5, 3)
print(x)

Construct a randomly initialized matrix:



In [ ]:
x = torch.rand(5, 3)
print(x)

Construct a matrix filled zeros and of dtype long:



In [ ]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

Construct a tensor directly from data:



In [ ]:
x = torch.tensor([5.5, 3])
print(x)

or create a tensor based on an existing tensor. These methods
will reuse properties of the input tensor, e.g. dtype, unless
new values are provided by user



In [ ]:
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)                                      # result has the same size

Get its size:



In [ ]:
print(x.size())

<div class="alert alert-info"><h4>Note</h4><p>``torch.Size`` is in fact a tuple, so it supports all tuple operations.</p></div>

## Operations

There are multiple syntaxes for operations. In the following
example, we will take a look at the addition operation.

Addition: syntax 1



In [ ]:
y = torch.rand(5, 3)
print(x + y)

Addition: syntax 2



In [ ]:
print(torch.add(x, y))

Addition: providing an output tensor as argument



In [ ]:
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

Addition: in-place



In [ ]:
# adds x to y
y.add_(x)
print(y)

<div class="alert alert-info"><h4>Note</h4><p>Any operation that mutates a tensor in-place is post-fixed with an ``_``.
    For example: ``x.copy_(y)``, ``x.t_()``, will change ``x``.</p></div>

You can use standard NumPy-like indexing with all bells and whistles!



In [ ]:
print(x[:, 1])

Resizing: If you want to resize/reshape tensor, you can use ``torch.view``:



In [ ]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

If you have a one element tensor, use ``.item()`` to get the value as a
Python number



In [ ]:
x = torch.randn(1)
print(x)
print(x.item())

**Read later:**


  100+ Tensor operations, including transposing, indexing, slicing,
  mathematical operations, linear algebra, random numbers, etc.,
  are described [here](https://pytorch.org/docs/torch).

## NumPy Bridge


Converting a Torch Tensor to a NumPy array and vice versa is a breeze.

The Torch Tensor and NumPy array will share their underlying memory
locations (if the Torch Tensor is on CPU), and changing one will change
the other.

### Converting a Torch Tensor to a NumPy Array



In [ ]:
a = torch.ones(5)
print(a)

In [ ]:
b = a.numpy()
print(b)

See how the numpy array changed in value.



In [ ]:
a.add_(1)
print(a)
print(b)

### Converting NumPy Array to Torch Tensor

See how changing the np array changed the Torch Tensor automatically

In [ ]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

All the Tensors on the CPU except a CharTensor support converting to
NumPy and back.

## CUDA Tensors

Tensors can be moved onto any device using the ``.to`` method.

The default device is 'cpu':

In [ ]:
b.device

But if you have a GPU (for example because you run this notebook on [Google Colab](https://colab.research.google.com/)), you can use the 'cuda' device:

In [ ]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

# Autograd: Automatic Differentiation


Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our
first model.


When executing tensor operations, PyTorch can automatically construct on-the-fly the graph of operations to compute the gradient of any quantity with respect to any tensor involved.

Let us see this in more simple terms with some examples.

## Tensor

``torch.Tensor`` is the central class of the package. If you set its attribute
``.requires_grad`` as ``True``, it starts to track all operations on it. When
you finish your computation you can call ``.backward()`` and have all the
gradients computed automatically. The gradient with respect to a tensor will be
accumulated into its ``.grad`` attribute.

To stop a tensor from tracking history, you can call ``.detach()`` to detach
it from the computation history, and to prevent future computation from being
tracked.

To prevent tracking history (and using memory), you can also wrap the code block
in ``with torch.no_grad():``. This can be particularly helpful when evaluating a
model because the model may have trainable parameters with
``requires_grad=True``, but for which we don't need the gradients


In [ ]:
x = torch.ones(2, 2)
print(x)

A Tensor has a Boolean field `requires_grad`, set to `False` by default, which states if PyTorch should build the graph of operations so that gradients with respect to it can be computed.

In [ ]:
print(x.requires_grad)
x.requires_grad_(True)
print(x.requires_grad)

You cannot call `numpy()` on a Tensor that requires grad, you first need to call `detach()`:

In [ ]:
x.detach().numpy()

Do a tensor operation:



In [ ]:
y = x + 2
print(y)

In [ ]:
y.requires_grad

Do more operations:



In [ ]:
z = y * y * 3
out = z.mean()

print(z) 
print(out)

After the computation is finished, i.e. the 'forward pass', you can call ```.backward()``` and have all the gradients computed automatically.

In [ ]:
out.backward()

The gradient with respect to a variable is accumulated into the ```.grad``` attribute:

In [ ]:
print(x.grad)

Let’s call the ``out``
*Variable* “$o$”.
We have that:

$y_i = x_i+2$

$z_i = 3 y_i^2$ 

$o = \frac{1}{4}\sum_i z_i$ 

**Forward pass:**

$y_i\bigr\rvert_{x_i=1} = 3$

$z_i\bigr\rvert_{y_i=3} = 27$

$o\bigr\rvert_{z_i=27} = 27$.

Taking derivatives give:

$\frac{d o}{d z_i} = \frac{1}{4}$

$\frac{d z_i}{d y_i} = 6 y_i$

$\frac{d y_i}{d x_i} =1$


hence by the **chain-rule:**

$\frac{d o}{d x_i}\bigr\rvert_{x_i=1} = \frac{d o}{d z_i}\bigr\rvert_{z_i=27}\frac{d z_i}{d y_i}\bigr\rvert_{y_i=3}\frac{d y_i}{d x_i}\bigr\rvert_{x_i=1} = \frac{1}{4} * 18 * 1 = 4.5$.

Can we do the same for 'y'?

In [ ]:
print(y.grad)

By default, gradients are only retained for leaf variables in the computational graph. Intermediate variables’ gradients are not retained to be inspected later. This was done by design, to save memory. However, you can enforce this behaviour by setting `retain_graph` to `True` when calling `backward`.

Let's reset our computations:

In [ ]:
x = torch.ones(2, 2)
x.requires_grad_(True)
y = x+2
z = 3 * y ** 2 
out = z.mean()

out.backward(retain_graph=True)

In [ ]:
print(x.grad)

To access the gradient with respect to intermediate (i.e. non-leaf) variables we have to use ```torch.autograd.grad```

In [ ]:
torch.autograd.grad(out, z, retain_graph=True)

In [ ]:
torch.autograd.grad(out, y, retain_graph=True)

Let's check the gradient with respect to 'x', and do a few more backward pass:

In [ ]:
print(x.grad)

In [ ]:
out.backward(retain_graph=True)

In [ ]:
print(x.grad)

In [ ]:
out.backward(retain_graph=True)

In [ ]:
print(x.grad)

The gradients must be set to zero manually. Otherwise they will cumulate across several backward calls. 
This accumulating behavior is desirable in particular to compute the gradient of a sum over multiple examples in a dataset.


In [ ]:
x.grad.data.zero_()

In [ ]:
out.backward()

In [ ]:
print(x.grad)